In [6]:
import os

import pandas as pd
from mistralai import Mistral
os.environ["MISTRAL_API_KEY"] = None
api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"
client = Mistral(api_key=api_key)
from src.mistral_trial import predict

ModuleNotFoundError: No module named 'mistralai'

In [41]:
import os
import pandas as pd
print(os.getcwd())
import re
import string
from collections import defaultdict
from src.utils_data import load_data


/home/waissa/recherche/projet


In [37]:
labels = {'Mot difficile ou inconnu',
          'Graphie, problème de déchiffrage',
          'Figure de style, expression idiomatique',
          'Référence culturelle difficile',
          'Difficulté liée à la grammaire',
          "Trop d'informations secondaires",
          "Indice de cohésion difficile",
          'Ordre syntaxique inhabituel',
          #'Autre',
}

In [28]:
global_df, local_df = load_data('.')

Index(['text', 'annotations'], dtype='object')


In [29]:
global_df.count()

text                   461
gold_score_20_label    461
classe                 461
dtype: int64

In [30]:
local_df = local_df.set_index('text_indice')

In [31]:
local_df

,text,annotations,gold_score_20_label,classe
text_indice,,,,
422,« Sortir du syndrome Tarzan » La brigade an...,"[{'text': 'collectif', 'start': 209, 'end': 21...",Accessible,B1
1735,Les cultures en Afrique du Nord Les trois p...,"[{'text': 'céréales', 'start': 388, 'end': 396...",Facile,A2
2084,Bienvenue à Gardanne Gardanne est une ville...,"[{'text': 'département', 'start': 62, 'end': 7...",Facile,A2
438,Le 8 octobre c’est la première journée paralym...,"[{'text': 'Andrew Parsons,', 'start': 611, 'en...",Facile,A2
133,Une destination à conseiller pour les vacances...,"[{'text': 'conseils', 'start': 644, 'end': 652...",Accessible,B1
...,...,...,...,...
1181,Démonstration Ikebana 1 variation et 1 créa...,"[{'text': 'Démonstration', 'start': 0, 'end': ...",Facile,A2
942,Une drôle de journée Et me voilà repartie p...,"[{'text': 'Flûte!', 'start': 792, 'end': 798, ...",Facile,A2
172,"L'adolescence ailleurs En Amérique, en Asie...","[{'text': 'L'adolescence', 'start': 0, 'end': ...",Facile,A2


In [32]:
predictions_df = pd.read_csv('./src/predictions.csv', sep='\t', index_col='text_indice')
predictions_df

,text,Mot difficile ou inconnu,"Graphie, problème de déchiffrage","Figure de style, expression idiomatique",Référence culturelle difficile,Difficulté liée à la grammaire,Trop d'informations secondaires,Indice de cohésion difficile,Ordre syntaxique inhabituel
text_indice,,,,,,,,,
422,« Sortir du syndrome Tarzan » La brigade an...,**négrophobie**\n**collectif**\n**antiraciste*...,**occupé**\n**démission**\n**Assemblée**\n**Ma...,**[Sortir du syndrome Tarzan]**\n**[le syndrom...,**Tarzan**\n**Canal+**\n**Marc Olivier Fogiel*...,**[Il] existe**\n**[Puis il y a eu]**\n**[Cela...,**Qu'est-ce que la négrophobie?** Tout ce qui...,- **Il** existe depuis 2005.\n- Puis **il** y ...,**Pourquoi les Noirs et les Arabes sont plus t...
1735,Les cultures en Afrique du Nord Les trois p...,**Maghreb**\n\n**agrumes**\n\n**oliviers**\n\n...,**Maghreb**\n**agrumes**\n**oliviers**\n**datt...,**Plus on va vers**\n**en raison du manque**\n...,**Maghreb**,**Plus on va**\n**deviennent**\n**en raison du...,"**quasiment**\n\n**Plus on va vers le sud, plu...","- Plus **on** va vers le sud, plus les culture...","**Plus on va vers le sud**\n**Ainsi, en Algéri..."
2084,Bienvenue à Gardanne Gardanne est une ville...,**[Autrefois]**\n**[Journées du Patrimoine]**,**Bouches**\n**Marché**\n**Provençale**\n**Sai...,**les hommes de Gardanne**\n**la mine de Garda...,**Bouches du Rhône**\n**Mine de Gardanne**\n**...,**travaillaient**\n**n’existe**\n**pourrez**,"**dans le département des Bouches du Rhône,**\...",- **Celle-ci**\n- **A**,"**Autrefois, les hommes de Gardanne travaillai..."
438,Le 8 octobre c’est la première journée paralym...,**Paralympique**\n**Comité**\n**Olympiques**\n...,**Paralympique**\n**Paralympiques**\n**Paralym...,**donner rendez-vous**\n**terrain de jeux**\n*...,**[Bastille]**\n**[Jeux Paralympiques]**\n**[C...,**A 2 ans**\n**sera**\n**Y aura**\n**sera là**...,**[A 2 ans des Jeux Paralympiques de Paris 202...,"- le 8 octobre, **La place de la Bastille vous...",**Le 8 octobre c’est la première journée paral...
133,Une destination à conseiller pour les vacances...,**[carnaval de Notting Hill]**\n**[sambas]**\n...,**2000**\n**Auteuil**\n**Mortagneau-Perche**\n...,**[avoir l’occasion]**\n**[un coin]**\n**[circ...,**Perche**\n**Mortagneau-Perche**\n**gîte**\n*...,**À 1 h 30**\n**n’avais jamais eu**\n**pique-n...,**[un pique-nique organisé par des amis dans c...,Une destination à conseiller pour les vacances...,"**À 1 h 30 de Paris, le Perche est une bonne d..."
...,...,...,...,...,...,...,...,...,...
1181,Démonstration Ikebana 1 variation et 1 créa...,**Ikebana**\n**ikebana**\n**variation**\n**Gra...,**VI°**\n**Ikebana**\n**ikebana**\n**Sogetsu**...,**faire vivre**\n**donner une seconde vie**\n*...,**Ikebana**\n\n**Sogetsu**\n\n**Hector Berlioz...,**Mardi 28 novembre à 18h30**\n\n**Conférence ...,"**Par Nicole Dineur, Grand Maître en ikebana, ...",- Il existe 3 grandes écoles. **Lors de cette ...,**arrivant au Japon de Chine par un moine japo...
942,Une drôle de journée Et me voilà repartie p...,**chargée comme une mule**\n**circulation**\n*...,**repairtie**\n\n**chargée**\n\n**envolé**\n\n...,**chargée comme une mule**\n**il ne s’était pa...,**comme une mule**,**s’était envolé**\n**s’était-il caché**\n**me...,**comme une mule**\n**Il ne s’était pas envolé...,Et me voilà repartie pour le supermarché! Je p...,**Et me voilà repartie pour le supermarché!**\...
172,"L'adolescence ailleurs En Amérique, en Asie...",**initiation**\n**archipel**\n**cocotiers**\n*...,**initiation**\n**Comores**\n**Madagascar**\n*...,**marqué par**\n**Perdu**\n**choyée**\n**convi...,**Houtsimou**\n**Masaï**,**est une étape essentielle**\n**est encore ma...,"**En Amérique, en Asie, en Afrique, ou sur les...","- Dans de nombreux pays, **ce** passage est en...","**Perdu dans l'océan Indien, entre Madagascar ..."


In [33]:
annotations_completes = pd.read_excel('./annotations_completes.xlsx')
annotations_completes['TermLabel'].value_counts()

TermLabel
Mot difficile ou inconnu                                        4310
Graphie, problème de déchiffrage                                2128
Figure de style, expression idiomatique                          914
Référence culturelle difficile                                   790
Indice de cohésion difficile (connecteur, pronom, inférence)     734
Ordre syntaxique inhabituel                                      561
Difficulté liée à la grammaire                                   469
Trop d'informations secondaires                                  371
Autre                                                            182
Name: count, dtype: int64

In [35]:
from collections import Counter

annotations = pd.read_json('annotations.json')
label_counter = Counter()

for i, row in annotations.iterrows():
    for a in row['annotations']:
        label_counter[a['label']] += 1

print(label_counter)

Counter({'Mot difficile ou inconnu': 3321, 'Graphie, problème de déchiffrage': 1905, 'Figure de style, expression idiomatique': 858, 'Référence culturelle difficile': 729, 'Indice de cohésion difficile (connecteur, pronom, inférence)': 671, 'Ordre syntaxique inhabituel': 553, 'Difficulté liée à la grammaire': 418, "Trop d'informations secondaires": 362, 'Autre': 168})


In [ ]:
no_annotations = []
for i, row in predictions_df.iterrows():
    if i not in local_df.index:
        no_annotations.append(i)
    else:
        for label in labels:
            print(i, local_df.loc[i]['classe'], label)
            text = row[label]
            matches = re.findall(r"\*\*(.*?)\*\*", text)
            cleaned = [re.sub(rf"[\[\]{re.escape(string.punctuation)}]", "", m).strip() for m in matches]
            print("prediciton", cleaned)
            print("annotations", [e['text'] for e in local_df.loc[i]['annotations'] if e['label']==label])


In [45]:
local_df

,text,annotations,gold_score_20_label,classe
text_indice,,,,
422,« Sortir du syndrome Tarzan » La brigade an...,"[{'text': 'collectif', 'start': 209, 'end': 21...",Accessible,B1
1735,Les cultures en Afrique du Nord Les trois p...,"[{'text': 'céréales', 'start': 388, 'end': 396...",Facile,A2
2084,Bienvenue à Gardanne Gardanne est une ville...,"[{'text': 'département', 'start': 62, 'end': 7...",Facile,A2
438,Le 8 octobre c’est la première journée paralym...,"[{'text': 'Andrew Parsons,', 'start': 611, 'en...",Facile,A2
133,Une destination à conseiller pour les vacances...,"[{'text': 'conseils', 'start': 644, 'end': 652...",Accessible,B1
...,...,...,...,...
1181,Démonstration Ikebana 1 variation et 1 créa...,"[{'text': 'Démonstration', 'start': 0, 'end': ...",Facile,A2
942,Une drôle de journée Et me voilà repartie p...,"[{'text': 'Flûte!', 'start': 792, 'end': 798, ...",Facile,A2
172,"L'adolescence ailleurs En Amérique, en Asie...","[{'text': 'L'adolescence', 'start': 0, 'end': ...",Facile,A2


In [57]:
results, no_annotations = compute_micro_per_class_metrics(predictions_df, local_df, labels)

In [55]:
results, no_annotations = compute_macro_per_class_metrics(predictions_df, local_df, labels)

In [49]:
import re
import string
from collections import defaultdict

def compute_micro_per_class_metrics(predictions_df, annotation_df, labels):
    """
    Compute precision, recall, and F1 per label, grouped by the 'classe' column of annotation_df.

    Args:
        predictions_df: DataFrame with predicted Markdown text per label.
        annotation_df: DataFrame with 'classe' and 'annotations' columns.
        labels: list of label types (e.g., ['date', 'location']).

    Returns:
        metrics: dict[class][label] = {Precision, Recall, F1, TP, FP, FN}
        no_annotations: list of example indices missing in annotation_df
    """
    metrics_per_class = defaultdict(lambda: defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0}))
    no_annotations = []

    for i, row in predictions_df.iterrows():
        if i not in annotation_df.index:
            no_annotations.append(i)
            continue

        true_class = annotation_df.loc[i]['classe']
        annotations = annotation_df.loc[i]['annotations']

        for label in labels:
            # Extract predicted parts (bolded using **)
            pred_text = row.get(label, "")
            matches = re.findall(r"\*\*(.*?)\*\*", pred_text)
            pred_parts = [re.sub(rf"[\[\]{re.escape(string.punctuation)}]", "", m).strip() for m in matches]

            # Extract ground truth parts for this label
            gt_parts = [a['text'] for a in annotations if a['label'] == label]

            pred_set = set(pred_parts)
            gt_set = set(gt_parts)

            tp = len(pred_set & gt_set)
            fp = len(pred_set - gt_set)
            fn = len(gt_set - pred_set)

            counts = metrics_per_class[true_class][label]
            counts['TP'] += tp
            counts['FP'] += fp
            counts['FN'] += fn

    # Compute metrics
    results = {}
    for cls, label_dict in metrics_per_class.items():
        results[cls] = {}
        for label, counts in label_dict.items():
            tp, fp, fn = counts['TP'], counts['FP'], counts['FN']
            precision = tp / (tp + fp) if (tp + fp) else 0.0
            recall = tp / (tp + fn) if (tp + fn) else 0.0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0

            results[cls][label] = {
                'Precision': precision,
                'Recall': recall,
                'F1 Score': f1,
                #'TP': tp,
                #'FP': fp,
                #'FN': fn
            }

    return results, no_annotations


In [53]:
def results_dict_to_df(results):
    rows = []
    for cls, label_dict in results.items():
        for label, metrics in label_dict.items():
            row = {
                'Class': cls,
                'Label': label,
                'Precision': metrics['Precision'],
                'Recall': metrics['Recall'],
                'F1 Score': metrics['F1 Score'],
            }
            rows.append(row)
    return pd.DataFrame(rows)

In [58]:
results_dict_to_df(results)

,Class,Label,Precision,Recall,F1 Score
0,B1,"Graphie, problème de déchiffrage",0.046144,0.091864,0.061430
1,B1,Mot difficile ou inconnu,0.177859,0.185449,0.181575
2,B1,"Figure de style, expression idiomatique",0.026734,0.063492,0.037625
3,B1,Ordre syntaxique inhabituel,0.004412,0.009772,0.006079
4,B1,Référence culturelle difficile,0.070323,0.132979,0.091996
5,B1,Difficulté liée à la grammaire,0.018309,0.221774,0.033825
6,B1,Indice de cohésion difficile,0.000000,0.000000,0.000000
7,B1,Trop d'informations secondaires,0.003195,0.029703,0.005769
8,A2,"Graphie, problème de déchiffrage",0.066474,0.145877,0.091330
9,A2,Mot difficile ou inconnu,0.126008,0.248262,0.167168


In [51]:
def compute_macro_per_class_metrics(predictions_df, annotation_df, labels):
    """
    Compute macro-averaged precision, recall, and F1 per label, grouped by 'classe'.

    Args:
        predictions_df: DataFrame with predicted Markdown text per label.
        annotation_df: DataFrame with 'classe' and 'annotations' columns.
        labels: list of label types (e.g., ['date', 'location']).

    Returns:
        metrics: dict[class][label] = macro-averaged {Precision, Recall, F1}
        no_annotations: list of example indices missing in annotation_df
    """
    per_class_label_metrics = defaultdict(lambda: defaultdict(list))
    no_annotations = []

    for i, row in predictions_df.iterrows():
        if i not in annotation_df.index:
            no_annotations.append(i)
            continue

        true_class = annotation_df.loc[i]['classe']
        annotations = annotation_df.loc[i]['annotations']

        for label in labels:
            # Extract predicted parts
            pred_text = row.get(label, "")
            matches = re.findall(r"\*\*(.*?)\*\*", pred_text)
            pred_parts = [re.sub(rf"[\[\]{re.escape(string.punctuation)}]", "", m).strip() for m in matches]

            # Extract ground truth parts for this label
            gt_parts = [a['text'] for a in annotations if a['label'] == label]

            pred_set = set(pred_parts)
            gt_set = set(gt_parts)

            tp = len(pred_set & gt_set)
            fp = len(pred_set - gt_set)
            fn = len(gt_set - pred_set)

            precision = tp / (tp + fp) if (tp + fp) else 0.0
            recall = tp / (tp + fn) if (tp + fn) else 0.0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0

            per_class_label_metrics[true_class][label].append((precision, recall, f1))

    # Average per class and per label
    results = {}
    for cls, label_dict in per_class_label_metrics.items():
        results[cls] = {}
        for label, values in label_dict.items():
            if values:
                precisions, recalls, f1s = zip(*values)
                results[cls][label] = {
                    'Precision': sum(precisions) / len(precisions),
                    'Recall': sum(recalls) / len(recalls),
                    'F1 Score': sum(f1s) / len(f1s),
                    'Count': len(values)
                }
            else:
                results[cls][label] = {
                    'Precision': 0.0,
                    'Recall': 0.0,
                    'F1 Score': 0.0,
                    'Count': 0
                }

    return results, no_annotations


In [33]:
row

text                                       Les cultures en Afrique du Nord    Les trois p...
Mot difficile ou inconnu                   **Maghreb**\n\n**agrumes**\n\n**oliviers**\n\n...
Graphie, problème de déchiffrage           **Maghreb**\n**agrumes**\n**oliviers**\n**datt...
Figure de style, expression idiomatique    **Plus on va vers**\n**en raison du manque**\n...
Référence culturelle difficile                                                   **Maghreb**
Difficulté liée à la grammaire             **Plus on va**\n**deviennent**\n**en raison du...
Trop d'informations secondaires            **quasiment**\n\n**Plus on va vers le sud, plu...
Indice de cohésion difficile               - Plus **on** va vers le sud, plus les culture...
Ordre syntaxique inhabituel                **Plus on va vers le sud**\n**Ainsi, en Algéri...
Name: 1735, dtype: object

In [25]:
row

text                                       « Sortir du syndrome Tarzan »    La brigade an...
Mot difficile ou inconnu                   **négrophobie**\n**collectif**\n**antiraciste*...
Graphie, problème de déchiffrage           **occupé**\n**démission**\n**Assemblée**\n**Ma...
Figure de style, expression idiomatique    **[Sortir du syndrome Tarzan]**\n**[le syndrom...
Référence culturelle difficile             **Tarzan**\n**Canal+**\n**Marc Olivier Fogiel*...
Difficulté liée à la grammaire             **[Il] existe**\n**[Puis il y a eu]**\n**[Cela...
Trop d'informations secondaires            **Qu'est-ce que la négrophobie?**  Tout ce qui...
Indice de cohésion difficile               - **Il** existe depuis 2005.\n- Puis **il** y ...
Ordre syntaxique inhabituel                **Pourquoi les Noirs et les Arabes sont plus t...
Name: 422, dtype: object

In [3]:
from jiwer import wer

reference = "hello world"
hypothesis = "hello duck"

error = wer(reference, hypothesis)
error

0.5

In [2]:
os.getcwd()

'/home/waissa/recherche/projet'

In [7]:
df = pd.read_csv('tmp.csv', sep='\t')
df

,text_indice,text,Mot difficile ou inconnu,"Graphie, problème de déchiffrage","Figure de style, expression idiomatique",Référence culturelle difficile,Difficulté liée à la grammaire,Trop d'informations secondaires,Indice de cohésion difficile,Ordre syntaxique inhabituel
0,422,« Sortir du syndrome Tarzan » La brigade an...,**négrophobie**\n**collectif**\n**antiraciste*...,**anti-négrophobie**\n**négrophobie**\n**faciè...,**contrôle au faciès**\n**Sortir du syndrome T...,**Tarzan**\n**Zyed et Bouna**\n**Canal+**\n**M...,**s'est formée**\n**ont eu lieu**\n**a posé**\...,**Pourquoi ce collectif ?**\n**Quelles sont se...,- Il existe depuis 2005. **Pourquoi ce collect...,**Pourquoi les Noirs et les Arabes sont plus t...
1,1735,Les cultures en Afrique du Nord Les trois p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2084,Bienvenue à Gardanne Gardanne est une ville...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,438,Le 8 octobre c’est la première journée paralym...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,133,Une destination à conseiller pour les vacances...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
456,1181,Démonstration Ikebana 1 variation et 1 créa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
457,942,Une drôle de journée Et me voilà repartie p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
458,172,"L'adolescence ailleurs En Amérique, en Asie...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
459,664,Je l'aime à mourir Moi je n'étais rien Et ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
predict("Qualtrics_Annotations_B.csv", "annotations_completes.xlsx", True, model, "tmp.csv")

Index(['text', 'annotations'], dtype='object')


  0%|          | 0/461 [00:22<?, ?it/s]


In [12]:
classes = [ "A1", "A2", "B1", "B2"]

In [ ]:
chat_response = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user",
            "content": "What is the best French cheese?",
        },
    ]
)
print(chat_response.choices[0].message.content)

### Classify

In [18]:
text = "Bien que la météo soit incertaine, nous avons décidé de partir en randonnée."
chat_response = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user",
            "content": (
                    f"Classifie la lisibilité du texte suivant dans une seule catégorie parmi : {', '.join(classes)}. \n"
                    "Répond uniquement avec le nom exact de la catégorie sans justification. \n"
                    f"Texte : {text} \nRéponse :"
                ),
        },
    ]
)
print(chat_response.choices[0].message.content)

B1


### Simplification

In [8]:
CEFR_LEVEL = "A2"  # Niveau de compétence en français
user_text = "Bien que la météo soit incertaine, nous avons décidé de partir en randonnée."  # Phrase complexe à simplifier

system_message = (
    f"Veuillez simplifier la phrase complexe suivante pour la rendre plus facile à lire et à comprendre "
    f"par des apprenants de français de niveau {CEFR_LEVEL} du CECRL.\n"
    f"Pour simplifier, vous pouvez remplacer les mots difficiles par des mots plus simples, élaborer ou "
    f"les supprimer lorsque cela est possible. Vous pouvez également diviser une phrase longue en phrases plus courtes et claires.\n"
    f"Assurez-vous que la phrase révisée est grammaticalement correcte, fluide et conserve le message "
    f"principal de l'original sans en changer le sens."
)

user_message = f"Phrase complexe : {user_text}"

chat_response = client.chat.complete(
    model=model,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(chat_response.choices[0].message.content)

Il fait peut-être mauvais, mais nous partons en randonnée.


### two turn conversation

In [13]:
CEFR_LEVEL = "A2"  # Niveau de compétence en français
user_text = "Bien que la météo soit incertaine, nous avons décidé de partir en randonnée."  # Phrase complexe à simplifier
assistant_message_1 = ""


# System message with instructions
system_message = (
    f"Veuillez simplifier la phrase complexe suivante pour la rendre plus facile à lire et à comprendre"
    f"par des apprenants de français de niveau {CEFR_LEVEL} du CECRL.\n"
    f"Pour simplifier, vous pouvez remplacer les mots difficiles par des mots plus simples, élaborer ou"
    f"les supprimer lorsque cela est possible. Vous pouvez également diviser une phrase longue en phrases plus courtes et claires.\n"
    f"Assurez-vous que la phrase révisée est grammaticalement correcte, fluide et conserve le message "
    f"principal de l'original sans en changer le sens."
)

# User message with the complex phrase
user_message_1 = f"Phrase complexe : {user_text}"

# First interaction: Send the initial user message and get the assistant's response
first_response = client.chat.complete(
    model=model,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message_1}
    ]
)

print(first_response)
# Extract the assistant's response from the first interaction
assistant_message_1 = first_response.choices[0].message.content

follow_up_question =  (f"Classifie la lisibilité du texte suivant dans une seule catégorie parmi : {', '.join(classes)}. \n "
                       f"Répond uniquement avec le nom exact de la catégorie sans justification. \n"
                       f"Texte : {assistant_message_1} \nRéponse :")



# Second interaction: Use the assistant's response as part of the input for the second message
second_response = client.chat.complete(
    model=model,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message_1},
        {"role": "assistant", "content": assistant_message_1},
        {"role": "user", "content": follow_up_question}
    ]
)

# Print the second response
print(second_response)

id='da6a42505e8643d7a9b7a16cafb60ccf' object='chat.completion' model='mistral-large-latest' usage=UsageInfo(prompt_tokens=178, completion_tokens=20, total_tokens=198) created=1746185991 choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content='Il fait peut-être mauvais, mais nous partons en randonnée.', tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')]
id='34bf388aa11d40f893b74e657a760c83' object='chat.completion' model='mistral-large-latest' usage=UsageInfo(prompt_tokens=286, completion_tokens=3, total_tokens=289) created=1746185992 choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content='A2', tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')]


In [17]:
assistant_message_1

'Il fait peut-être mauvais, mais nous partons en randonnée.'

In [16]:
second_response.choices[0].message.content

'A2'

### debug .py

In [2]:
from src.mistral_trial import *
import os

In [3]:
os.environ["MISTRAL_API_KEY"] = None
api_key = os.environ["MISTRAL_API_KEY"]
mistralai = True
model = "mistral-large-latest"
mistral_chat = Mistral(api_key=api_key)

In [5]:
text = "Le biplan survolait la région, tandis que les scientifiques étudiaient les effets de l'anthracite sur l'environnement."
reader_level = "B1"
difficult_words = identify_difficult_words(text, reader_level, mistralai, model, explication=False)
print(difficult_words)


**biplan**
**anthracite**


In [6]:
text = "La commande Y543278782543164 a été envoyée au XIV siècle, et 315000 personnes ont participé à l'initiative."
reader_level = "A2"
deciphering_issues = identify_deciphering_issues(text, reader_level, mistralai, model, explication=True)
print(deciphering_issues)

1. **Y543278782543164** : Ce nombre est composé d'une combinaison de lettres et de chiffres, ce qui peut rendre sa lecture et sa compréhension difficiles pour un lecteur de niveau A2. Cependant, il appartient à une autre catégorie : numératie.

2. **XIV** : L'utilisation de chiffres romains pour indiquer le siècle peut poser des difficultés de déchiffrage pour un lecteur de niveau A2, car cette notation n'est pas couramment utilisée dans la vie quotidienne et peut ne pas être familière.

3. **315000** : Ce nombre est écrit sans espaces, ce qui peut rendre sa lecture difficile. Pour un lecteur de niveau A2, il peut être difficile de comprendre immédiatement la valeur de ce nombre.

4. **initiative** : La graphie de ce mot peut poser des difficultés d’accès au sens, mais il reste connu à l’oral.


In [7]:
text = "Il n’en reste pas moins que l'hôtel accueillait dans ses murs des voyageurs fatigués par leur travail."
reader_level = "A2"
figurative_expressions_no_explication = identify_figurative_expressions(text, reader_level, mistralai, model, explication=False)
print(figurative_expressions_no_explication)

**il n’en reste pas moins que**
**accueillait dans ses murs**
**fatigués par leur travail**


In [8]:
text = "Venez à Bruxelles en Thalys. On a eu en même temps Moebius, Goscinny, Enki Bilal et plein d’autres."
reader_level = "A2"
cultural_references_no_explication = identify_cultural_references(text, reader_level, mistralai, model, explication=False)
print(cultural_references_no_explication)

**Thalys**
**Moebius**
**Goscinny**
**Enki Bilal**


In [9]:
text = "Je préfère me faire désirer, me faire attendre plutôt qu’attendre. Les tiges ayant souffert du gel."
reader_level = "A2"
grammatical_difficulties_no_explication = identify_grammatical_difficulties(text, reader_level, mistralai, model, explication=False)
print(grammatical_difficulties_no_explication)

**me faire désirer**
**me faire attendre**
**ayant souffert**


In [10]:
text = "Cette année, comme l’an prochain, comme depuis 1790, le défilé du 14 juillet nous rappellera que certaines choses méritent qu’on s’engage et qu’on se batte pour elles, que la paix n’est pas un confort qu’on achète par des concessions."
reader_level = "A2"
secondary_information_no_explication = identify_secondary_information(text, reader_level, mistralai, model, explication=False)
print(secondary_information_no_explication)

**comme l’an prochain, comme depuis 1790**
**que la paix n’est pas un confort qu’on achète par des concessions**


In [11]:
text = "Il faut un grand terrain, un pré ou une plage, et beaucoup de copains car il faut réunir deux équipes, de 7 sur l’herbe ou de 5 sur la plage."
reader_level = "A2"
cohesion_issues_no_explication = identify_cohesion_issues(text, reader_level, mistralai, model, explication=False)
print(cohesion_issues_no_explication)

Il faut un grand terrain, un pré ou une plage, et beaucoup de copains car il faut réunir deux équipes, **de 7 sur l’herbe ou de 5 sur la plage**.


In [12]:
text = "Là où vivaient des arbres maintenant la ville est là. Mourir vos beaux yeux, belle Marquise, d’amour me font."
reader_level = "A2"
unusual_syntax_no_explication = identify_unusual_syntax(text, reader_level, mistralai, model, explication=False)
print(unusual_syntax_no_explication)

**Là où vivaient des arbres maintenant la ville est là.**
**Mourir vos beaux yeux, belle Marquise, d’amour me font.**
